In [5]:
import os
import data_processing_tool as dpt
from datetime import timedelta, date, datetime
from args_parameter import args
from PrepareData import ACCESS_BARRA_v1,ACCESS_BARRA_v2

import torch,os,torchvision
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader,random_split
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim

# from PIL import Image
import time
from sklearn.model_selection import StratifiedShuffleSplit
import model
import utility
from tqdm import tqdm
import math
import xarray as xr
from skimage.measure import compare_ssim
from skimage.measure import compare_psnr,compare_mse

args.file_ACCESS_dir_pr="/g/data/ub7/access-s1/hc/raw_model/atmos/pr/daily/"
args.file_ACCESS_dir="/g/data/ub7/access-s1/hc/raw_model/atmos/"
# training_name="temp01"
args.file_BARRA_dir="/g/data/ma05/BARRA_R/v1/forecast/spec/accum_prcp/"
args.channels=0
if args.pr:
    args.channels+=1
if args.dem:
    args.channels+=1
if args.zg:
    args.channels+=1
if args.psl:
    args.channels+=1
if args.tasmax:
    args.channels+=1
if args.tasmin:
    args.channels+=1
access_rgb_mean= 2.9067910245780248e-05*86400

leading_time=217
args.leading_time_we_use=7
args.ensemble=2

init_date=date(1970, 1, 1)
start_date=date(1990, 1, 2)
# end_date=date(2012,12,25) #if 929 is true we should substract 1 day
dates=[start_date + timedelta(x) for x in range((end_date - start_date).days + 1)]
print(access_rgb_mean)

print("training statistics:")
print("  ------------------------------")
print("  trainning name  |  %s"%args.train_name)
print("  ------------------------------")
print("  num of channels | %5d"%args.channels)
print("  ------------------------------")
print("  num of threads  | %5d"%args.n_threads)
print("  ------------------------------")
print("  batch_size      | %5d"%args.batch_size)



2.5114674452354135
training statistics:
  ------------------------------
  trainning name  |  temp01
  ------------------------------
  num of channels |     1
  ------------------------------
  num of threads  |     8
  ------------------------------
  batch_size      |    16


In [15]:
#for debuging on my computer
import os
import data_processing_tool as dpt
from datetime import timedelta, date, datetime
from args_parameter import args
from PrepareData import ACCESS_BARRA_v1,ACCESS_BARRA_v2

import torch,os,torchvision
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader,random_split
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim

# from PIL import Image
import time
from sklearn.model_selection import StratifiedShuffleSplit
import model
import utility
from tqdm import tqdm
import math
import xarray as xr
from skimage.measure import compare_ssim
from skimage.measure import compare_psnr,compare_mse

args.file_ACCESS_dir="E:/climate/access-s1/"
# args.file_ACCESS_dir="H:/climate/access-s1/"

args.file_BARRA_dir="C:/Users/JIA059/barra/"
# args.file_BARRA_dir="D:/dataset/accum_prcp/"

args.channels=1
args.batch_size=16

# ensemble=['e01','e02']
args.ensemble=2
access_rgb_mean= 2.9067910245780248e-05*86400

leading_time=217
args.leading_time_we_use=7
# args.lr=0.001

init_date=date(1970, 1, 1)
start_date=date(1990, 1, 2)
end_date=date(1990,12,25) #if 929 is true we should substract 1 day
dates=[start_date + timedelta(x) for x in range((end_date - start_date).days + 1)]
print(access_rgb_mean)
print("training statistics:")
print("  ------------------------------")
print("  trainning name  |  %s"%args.train_name)
print("  ------------------------------")
print("  num of channels | %5d"%args.channels)
print("  ------------------------------")
print("  num of threads  | %5d"%args.n_threads)
print("  ------------------------------")
print("  batch_size      | %5d"%args.batch_size)


2.5114674452354135
training statistics:
  ------------------------------
  trainning name  |  temp01
  ------------------------------
  num of channels |     1
  ------------------------------
  num of threads  |     8
  ------------------------------
  batch_size      |    16


In [16]:
train_transforms = transforms.Compose([
#     transforms.Resize(IMG_SIZE),
#     transforms.RandomResizedCrop(IMG_SIZE),
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomRotation(30),
    transforms.ToTensor()
#     transforms.Normalize(IMG_MEAN, IMG_STD)
])

data_set=ACCESS_BARRA_v2(start_date,end_date,transform=train_transforms,args=args)
train_data,val_data=random_split(data_set,[int(len(data_set)*0.8),len(data_set)-int(len(data_set)*0.8)])


print("Dataset statistics:")
print("  ------------------------------")
print("  total | %5d"%len(data_set))
print("  ------------------------------")
print("  train | %5d"%len(train_data))
print("  ------------------------------")
print("  val   | %5d"%len(val_data))

=> BARRA_R & ACCESS_S1 loading
=> from 1990/01/02 to 1990/12/25
C:/Users/JIA059/barra/
no file or no permission!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Dataset statistics:
  ------------------------------
  total |   603
  ------------------------------
  train |   482
  ------------------------------
  val   |   121


In [17]:

train_dataloders =DataLoader(train_data,
                             batch_size=args.batch_size,
                             shuffle=False,
                             num_workers=0)
val_dataloders =DataLoader(val_data,
                           batch_size=args.batch_size,
                           shuffle=False,
                           num_workers=0)



In [18]:
args.cpu=True
# args.pre_train =False
# args.pre_train ="C:/Users/JIA059/climate_v1_csiro/High-resolution-seasonal-climate-forecast_v1_csiro/model/RCAN_BIX"+str(args.scale[0])+".pt"
# "C:/Users/JIA059/climate_v1_csiro/High-resolution-seasonal-climate-forecast_v1_csiro/model"
def prepare( l, volatile=False):
    device = torch.device('cpu' if args.cpu else 'cuda')
    def _prepare(tensor):
        if args.precision == 'half': tensor = tensor.half()
        return tensor.to(device)

    return [_prepare(_l) for _l in l]

checkpoint = utility.checkpoint(args)
net = model.Model(args, checkpoint).double()
args.lr=0.001
criterion = nn.L1Loss()
optimizer_my = optim.SGD(net.parameters(), lr=args.lr, momentum=0.9)
# scheduler = optim.lr_scheduler.StepLR(optimizer_my, step_size=7, gamma=0.1)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer_my, gamma=0.9)
# torch.optim.lr_scheduler.MultiStepLR(optimizer_my, milestones=[20,80], gamma=0.1)

Making model...
accesss-s1 mean (0.4690)


In [1]:
import os
from importlib import import_module

import torch
import torch.nn as nn
from torch.autograd import Variable

class Model(nn.Module):
    def __init__(self, args, ckp):
        super(Model, self).__init__()
        print('Making model...')

        self.scale = args.scale
        self.idx_scale = 0
        self.self_ensemble = args.self_ensemble
        self.chop = args.chop
        self.precision = args.precision
        self.cpu = args.cpu
        self.device = torch.device('cpu' if args.cpu else 'cuda')
        self.n_GPUs = args.n_GPUs
        self.save_models = args.save_models

        module = import_module('model.' + args.model.lower())
        self.model = module.make_model(args).to(self.device)
        if args.precision == 'half': self.model.half()

        if not args.cpu and args.n_GPUs > 1:
            ckp.my_write_log("Let's use"+str(torch.cuda.device_count())+"GPUs!")
            self.model = nn.DataParallel(self.model, range(args.n_GPUs))

        self.load(
            ckp.dir,
            pre_train=args.pre_train,
            resume=args.resume,
            cpu=args.cpu
        )
        if args.print_model: print(self.model)

    def forward(self, x, idx_scale):
        self.idx_scale = idx_scale
        target = self.get_model()
        if hasattr(target, 'set_scale'):
            target.set_scale(idx_scale)

        if self.self_ensemble and not self.training:
            if self.chop:
                forward_function = self.forward_chop
            else:
                forward_function = self.model.forward

            return self.forward_x8(x, forward_function)
        elif self.chop and not self.training:
            return self.forward_chop(x)
        else:
            return self.model(x)

    def get_model(self):
        if self.n_GPUs == 1:
            return self.model
        else:
            return self.model
            return self.model.module

    def state_dict(self, **kwargs):
        target = self.get_model()
        return target.state_dict(**kwargs)

    def save(self, apath, epoch, is_best=False):
        target = self.get_model()
        torch.save(
            target.state_dict(), 
            os.path.join(apath, 'model', 'model_latest.pt')
        )
        if is_best:
            torch.save(
                target.state_dict(),
                os.path.join(apath, 'model', 'model_best.pt')
            )
        else:
            torch.save(
                target.state_dict(),
                os.path.join(apath, 'model', 'model_{}.pt'.format(epoch))
            )
        
        if self.save_models:
            torch.save(
                target.state_dict(),
                os.path.join(apath, 'model', 'model_{}.pt'.format(epoch))
            )

    def load(self, apath, pre_train='.', resume=-1, cpu=False):
        if cpu:
            kwargs = {'map_location': lambda storage, loc: storage}
        else:
            kwargs = {}

        if resume == -1:
            self.get_model().load_state_dict(
                torch.load(
                    os.path.join(apath, 'model', 'model_latest.pt'),
                    **kwargs
                ),
                strict=False
            )
        elif resume == 0:
            if pre_train != '.':
                print('Loading model from {}'.format(pre_train))
                self.get_model().load_state_dict(
                    torch.load(pre_train, **kwargs),
                    strict=False
                )
        else:
            self.get_model().load_state_dict(
                torch.load(
                    os.path.join(apath, 'model', 'model_{}.pt'.format(resume)),
                    **kwargs
                ),
                strict=False
            )
    # shave = 10, min_size=160000
    def forward_chop(self, x, shave=10, min_size=160000):
        scale = self.scale[self.idx_scale]
        n_GPUs = min(self.n_GPUs, 4)
        b, c, h, w = x.size()
        h_half, w_half = h // 2, w // 2
        h_size, w_size = h_half + shave, w_half + shave
        lr_list = [
            x[:, :, 0:h_size, 0:w_size],
            x[:, :, 0:h_size, (w - w_size):w],
            x[:, :, (h - h_size):h, 0:w_size],
            x[:, :, (h - h_size):h, (w - w_size):w]]

        if w_size * h_size < min_size:
            sr_list = []
            for i in range(0, 4, n_GPUs):
                lr_batch = torch.cat(lr_list[i:(i + n_GPUs)], dim=0)
                sr_batch = self.model(lr_batch)
                sr_list.extend(sr_batch.chunk(n_GPUs, dim=0))
        else:
            sr_list = [
                self.forward_chop(patch, shave=shave, min_size=min_size) \
                for patch in lr_list
            ]

        h, w = scale * h, scale * w
        h_half, w_half = scale * h_half, scale * w_half
        h_size, w_size = scale * h_size, scale * w_size
        shave *= scale

        output = x.new(b, c, h, w)
        output[:, :, 0:h_half, 0:w_half] \
            = sr_list[0][:, :, 0:h_half, 0:w_half]
        output[:, :, 0:h_half, w_half:w] \
            = sr_list[1][:, :, 0:h_half, (w_size - w + w_half):w_size]
        output[:, :, h_half:h, 0:w_half] \
            = sr_list[2][:, :, (h_size - h + h_half):h_size, 0:w_half]
        output[:, :, h_half:h, w_half:w] \
            = sr_list[3][:, :, (h_size - h + h_half):h_size, (w_size - w + w_half):w_size]

        return output

    def forward_x8(self, x, forward_function):
        def _transform(v, op):
            if self.precision != 'single': v = v.float()

            v2np = v.data.cpu().numpy()
            if op == 'v':
                tfnp = v2np[:, :, :, ::-1].copy()
            elif op == 'h':
                tfnp = v2np[:, :, ::-1, :].copy()
            elif op == 't':
                tfnp = v2np.transpose((0, 1, 3, 2)).copy()

            ret = torch.Tensor(tfnp).to(self.device)
            if self.precision == 'half': ret = ret.half()

            return ret

        lr_list = [x]
        for tf in 'v', 'h', 't':
            lr_list.extend([_transform(t, tf) for t in lr_list])

        sr_list = [forward_function(aug) for aug in lr_list]
        for i in range(len(sr_list)):
            if i > 3:
                sr_list[i] = _transform(sr_list[i], 't')
            if i % 4 > 1:
                sr_list[i] = _transform(sr_list[i], 'h')
            if (i % 4) % 2 == 1:
                sr_list[i] = _transform(sr_list[i], 'v')

        output_cat = torch.cat(sr_list, dim=0)
        output = output_cat.mean(dim=0, keepdim=True)

        return output



In [13]:
#training


max_error=np.inf
for e in range(args.epochs):
    #train
    net.train()
    loss=0
    start=time.time()
#     if e % 10 == 0:
#         for p in optimizer_my.param_groups:
#             p['lr'] *= 0.9

    for batch, (lr, hr,_,_) in enumerate(train_dataloders):
    #     print(batch, (lr.size(), hr.size()))
        lr, hr = prepare([lr, hr])
        optimizer_my.zero_grad()
        with torch.set_grad_enabled(True):
            sr = net(lr, 0)
#         error = criterion(sr[:,:,:,0:403], hr)
            running_loss =criterion(sr, hr)
            loss+=running_loss #.copy()?
        running_loss.backward()
        optimizer_my.step()
        
    #validation
    net.eval()
    start=time.time()
    with torch.no_grad():
        eval_psnr=0
        eval_ssim=0
        tqdm_val = tqdm(val_dataloders, ncols=80)
        for idx_img, (lr, hr,date,_) in enumerate(tqdm_val):
            lr, hr = prepare([lr, hr])
            sr = net(lr, 0)
            val_loss=criterion(sr, hr)
            for ssr,hhr in zip(sr,hr):
                eval_psnr+=compare_psnr(ssr[0].cpu().numpy(),hhr[0].cpu().numpy(),data_range=(hhr[0].cpu().max()-hhr[0].cpu().min()).item() )
                eval_ssim+=compare_ssim(ssr[0].cpu().numpy(),hhr[0].cpu().numpy(),data_range=(hhr[0].cpu().max()-hhr[0].cpu().min()).item() )      
#             print("psnr: %f "%(eval_psnr/len(test_data)))
    print("epoche: %d,time cost %f s, lr: %f, train_loss: %f,validation loss:%f "%(
              e,
              time.time()-start,
              optimizer_my.state_dict()['param_groups'][0]['lr'],
              loss.item()/len(train_data),
              val_loss
         ))
    if running_loss<max_error:
        max_error=running_loss
#         torch.save(net,train_loss"_"+str(e)+".pkl")
        if not os.path.exists("./model/save/"+args.train_name+"/"):
            os.mkdir("./model/save/"+args.train_name+"/")
        torch.save(net,"./model/save/"+args.train_name+"/"+str(e)+".pkl")
    


end loading one data,time cost 2.676250
end loading one data,time cost 3.161077


KeyboardInterrupt: 

In [ ]:
args.train_name="temp02"
if not os.path.exists("./model/save/"+args.train_name+"/"):
    os.mkdir("./model/save/"+args.train_name+"/")
# torch.save(net,"./model/save/"+training_name+"/"+str(e)+".pkl")

In [ ]:
#testing,and evaluation
from skimage.measure import compare_ssim
from skimage.measure import compare_psnr,compare_mse




net.eval()
start=time.time()
with torch.no_grad():
    eval_psnr=0
    eval_ssim=0
    tqdm_val = tqdm(val_dataloders, ncols=80)
    for idx_img, (lr, hr,date,_) in enumerate(tqdm_val):
        lr, hr = prepare([lr, hr])
        sr = net(lr, 0)
        val_loss=criterion(sr, hr)
        for ssr,hhr in zip(sr,hr):
            eval_psnr+=compare_psnr(ssr[0].cpu().numpy(),hhr[0].cpu().numpy(),data_range=(hhr[0].cpu().max()-hhr[0].cpu().min()).item() )
            eval_ssim+=compare_ssim(ssr[0].cpu().numpy(),hhr[0].cpu().numpy(),data_range=(hhr[0].cpu().max()-hhr[0].cpu().min()).item() )
            
            
            
        break
#     break
            
    print("psnr: %f "%(eval_psnr/len(val_data)))
        
print("time cost: %f s "%(time.time()-start))

In [ ]:
# (len(test_dataloders)*args.batch_size )
print(0+val_loss)

In [ ]:
(hr.max()-hr.min()).item()
print("psnr: %f "%(hr.max()-hr.min()).item() )
ssim_avg=0
psnr_avg=0
for ssr,hhr in zip(sr,hr):
#     print(hhr[0].shape)
#     print(compare_ssim(ssr[0].numpy(),hhr[0].numpy(),data_range=(hhr[0].max()-hhr[0].min()).item() ))
    psnr_avg+=compare_psnr(ssr[0].numpy(),hhr[0].numpy(),data_range=(hhr[0].max()-hhr[0].min()).item() )
    ssim_avg+=compare_ssim(ssr[0].numpy(),hhr[0].numpy(),data_range=(hhr[0].max()-hhr[0].min()).item() )
print(ssim_avg/args.batch_size)
print(psnr_avg/args.batch_size)
compare_psnr(ssr[0].numpy(),hhr[0].numpy(),data_range=(hhr[0].max()-hhr[0].min()).item() )

print(args.batch_size)

In [ ]:
print(sr.shape,hr.shape)
compare_psnr(hhr[0].numpy(),hhr[0].numpy(),data_range=(hhr[0].max()-hhr[0].min()).item() )
print(len(test_data))
print(len(test_dataloders))
# data_set.lon.data

In [ ]:
# demo:



with torch.no_grad():
    import time
    start=time.time()
    print("test time for sigle day %s " %(time.time()-start))

    sr_numpy=sr.numpy()[7][0]
#     sr_numpy=np.ascontiguousarray(sr_numpy.transpose((1, 0)))
    data_sr=xr.DataArray(sr_numpy,coords=[data_set.lat.data,data_set.lon.data],dims=["lat","lon"])
    
    hr_numpy=hr.numpy()[7][0]
#     hr_numpy=np.ascontiguousarray(hr_numpy.transpose((1, 0)))
    data_hr=xr.DataArray(hr_numpy,coords=[data_set.lat.data,data_set.lon.data],dims=["lat","lon"])
    print(date)
    dpt.draw_aus(data_hr,
                 colormap = plt.cm.get_cmap('RdBu_r', 10),
                 title="super resolution we predicted",
                 save=False)
    dpt.draw_aus(data_sr,
                 colormap = plt.cm.get_cmap('RdBu_r', 10),
                 title="super resolution we predicted",
                 save=False)

# psnr1(sr, hr)
# print(hr.max())

In [ ]:
print(hr_numpy.shape)
print(data_set.lon.data.shape)
print(data_set.lat.data.shape)

In [ ]:
#testing



epoch = self.scheduler.last_epoch + 1
self.ckp.write_log('\nEvaluation:')
self.ckp.add_log(torch.zeros(1, len(self.scale)))
self.model.eval()

timer_test = utility.timer()
with torch.no_grad():
    for idx_scale, scale in enumerate(self.scale):
        eval_acc = 0
        self.loader_test.dataset.set_scale(idx_scale)
        tqdm_test = tqdm(self.loader_test, ncols=80)
        for idx_img, (lr, hr, filename, _) in enumerate(tqdm_test):
            filename = filename[0]
            no_eval = (hr.nelement() == 1)
            if not no_eval:
                lr, hr = self.prepare([lr, hr])
            else:
                lr = self.prepare([lr])[0]

            sr = self.model(lr, idx_scale)
            sr = utility.quantize(sr, self.args.rgb_range)

            save_list = [sr]
            if not no_eval:
                eval_acc += utility.calc_psnr(
                    sr, hr, scale, self.args.rgb_range,
                    benchmark=self.loader_test.dataset.benchmark
                )
                save_list.extend([lr, hr])

            if self.args.save_results:
                self.ckp.save_results(filename, save_list, scale)

        self.ckp.log[-1, idx_scale] = eval_acc / len(self.loader_test)
        best = self.ckp.log.max(0)
        self.ckp.write_log(
            '[{} x{}]\tPSNR: {:.3f} (Best: {:.3f} @epoch {})'.format(
                self.args.data_test,
                scale,
                self.ckp.log[-1, idx_scale],
                best[0][idx_scale],
                best[1][idx_scale] + 1
            )
        )

self.ckp.write_log(
    'Total time: {:.2f}s\n'.format(timer_test.toc()), refresh=True
)
if not self.args.test_only:
    self.ckp.save(self, epoch, is_best=(best[1][0] + 1 == epoch))

In [ ]:
torch.new()
np.random.rand(8, 4, 78, 100)

In [ ]:

    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))



# channel=1
# x=np.random.rand(8,channel, 78, 100)
# x=x.astype(np.float32)
# print(x.dtype)

# means, stdevs = [], []

# for i in range(channel):
#     pixels = x[:, i, :, :].ravel()  # 拉成一行
#     means.append(np.mean(pixels))
#     stdevs.append(np.std(pixels))

# x=torch.tensor(x)


# # std=np.std(x.ravel(),axis=0)
# # print(std.shape)
# # mean=np.mean(x)
# print(means)
# print(stdevs)

# # x=torch.rand((8, 3, 78, 100))
# # std=torch.std(x)
# # mean=torch.mean(x)

# # x.dtype="float32"

# class MeanShift(nn.Conv2d):
#     def __init__(self, rgb_range, rgb_mean, rgb_std,channels, sign=-1):
#         super(MeanShift, self).__init__(channel, channel, kernel_size=1)
#         std = torch.Tensor(rgb_std)
#         self.weight.data = torch.eye(channel).view(channel, channel, 1, 1)
#         self.weight.data.div_(std.view(channel, 1, 1, 1))
#         self.bias.data = sign * rgb_range * torch.Tensor(rgb_mean)
#         self.bias.data.div_(std)
#         self.requires_grad = False
        
# aa=MeanShift(1,means,stdevs,3)
# # aa(x)
# np.mean(aa(x).detach().numpy())


In [ ]:
# # checkpoint = utility.checkpoint(args)
# # net = model.Model(args, checkpoint)
# # criterion = nn.L1Loss()
# # optimizer_my = optim.SGD(net.parameters(), lr=0.00001, momentum=0.9)
# def prepare( l, volatile=False):
#     device = torch.device('cpu' if args.cpu else 'cuda')
#     def _prepare(tensor):
#         if args.precision == 'half': tensor = tensor.half()
#         return tensor.to(device)

#     return [_prepare(_l) for _l in l]

# max_error=10000
# for e in range(args.epochs):
#     if e % 10 == 0:
#         for p in optimizer_my.param_groups:
#             p['lr'] *= 0.9
        
#     net.train()
#     for batch, (lr, hr) in enumerate(train_dataloders):
#         lr, hr = prepare([lr, hr])
# #         print(lr)

#         optimizer_my.zero_grad()
#         sr = net(lr, 0)
# #         sr=utility.fit_size(sr,args)

#         error = criterion(sr[:,:,:,0:403], hr)
#         if error<max_error:
#             max_error=error
#             torch.save(net,"C:/Users/JIA059/climate_v1_csiro/High-resolution-seasonal-climate-forecast_v1_csiro/model/save/"+str(e)+".pkl")
#         error.backward()
#         optimizer_my.step()
#     print("epoche: %d, lr: %f, error: %f"%(e,optimizer_my.state_dict()['param_groups'][0]['lr'],error.item()))

In [ ]:
end loading one data,time cost 1.715966
end loading one data,time cost 1.632074
end loading one data,time cost 1.671565
end loading one data,time cost 1.715903
end loading one data,time cost 1.605141
end loading one data,time cost 1.621977
end loading one data,time cost 1.621583
end loading one data,time cost 1.621500
end loading one data,time cost 1.600586
end loading one data,time cost 1.594560
end loading one data,time cost 1.593534
end loading one data,time cost 1.604488
end loading one data,time cost 1.606978
end loading one data,time cost 1.600660
end loading one data,time cost 1.600594
end loading one data,time cost 1.605244
end loading one data,time cost 1.692404
end loading one data,time cost 1.619076
end loading one data,time cost 1.682503
end loading one data,time cost 1.620104
end loading one data,time cost 1.628786
end loading one data,time cost 1.607090
end loading one data,time cost 1.670968
end loading one data,time cost 1.671936
end loading one data,time cost 1.602363
end loading one data,time cost 1.654448
end loading one data,time cost 1.673330
end loading one data,time cost 1.623322
end loading one data,time cost 1.635231
end loading one data,time cost 1.642935
end loading one data,time cost 1.650784
end loading one data,time cost 1.645927
end loading one data,time cost 1.726419
end loading one data,time cost 1.690695
end loading one data,time cost 1.689773
end loading one data,time cost 1.711888
end loading one data,time cost 1.612949
end loading one data,time cost 1.617190
end loading one data,time cost 1.640625
end loading one data,time cost 1.674064
end loading one data,time cost 1.642350
end loading one data,time cost 1.647243
end loading one data,time cost 1.606059
end loading one data,time cost 1.593960
end loading one data,time cost 1.629991
end loading one data,time cost 1.602437
end loading one data,time cost 1.647727
end loading one data,time cost 1.660140
end loading one data,time cost 1.712363
end loading one data,time cost 1.706504
end loading one data,time cost 1.684479
end loading one data,time cost 1.705693
end loading one data,time cost 1.656246
end loading one data,time cost 1.632491
end loading one data,time cost 1.641997
end loading one data,time cost 1.640489
end loading one data,time cost 1.615456
end loading one data,time cost 1.657358
end loading one data,time cost 1.619042
end loading one data,time cost 1.591613
end loading one data,time cost 1.642784
end loading one data,time cost 1.649220
end loading one data,time cost 1.614412
end loading one data,time cost 1.629373
end loading one data,time cost 1.681119
end loading one data,time cost 1.661355
end loading one data,time cost 1.656491
end loading one data,time cost 1.663970
end loading one data,time cost 1.660205
end loading one data,time cost 1.646195
end loading one data,time cost 1.666683
end loading one data,time cost 1.668686
end loading one data,time cost 1.625349
end loading one data,time cost 1.636791
end loading one data,time cost 1.644629
end loading one data,time cost 1.653769
end loading one data,time cost 1.667453
end loading one data,time cost 1.653591
end loading one data,time cost 1.660961
end loading one data,time cost 1.613453
end loading one data,time cost 1.696342
end loading one data,time cost 1.649550
end loading one data,time cost 1.633617
end loading one data,time cost 1.665265
end loading one data,time cost 1.657074
end loading one data,time cost 1.632226
end loading one data,time cost 1.695935
end loading one data,time cost 1.654280
end loading one data,time cost 1.635847
end loading one data,time cost 1.608618
end loading one data,time cost 1.642667
end loading one data,time cost 1.651407
end loading one data,time cost 1.668395
end loading one data,time cost 1.671665
end loading one data,time cost 1.657387
end loading one data,time cost 1.668278
end loading one data,time cost 1.723833
end loading one data,time cost 1.661024
end loading one data,time cost 1.662430
end loading one data,time cost 1.684431
end loading one data,time cost 1.645400
end loading one data,time cost 1.617971
end loading one data,time cost 1.611092
end loading one data,time cost 1.622975
end loading one data,time cost 1.649650
end loading one data,time cost 1.641810
end loading one data,time cost 1.656899
end loading one data,time cost 1.656220
end loading one data,time cost 1.665733
end loading one data,time cost 1.688295
end loading one data,time cost 1.658993
end loading one data,time cost 1.674594
end loading one data,time cost 1.703684
end loading one data,time cost 1.704101
end loading one data,time cost 1.638448
end loading one data,time cost 1.674996
end loading one data,time cost 1.631276
end loading one data,time cost 1.634202
end loading one data,time cost 1.636754
end loading one data,time cost 1.702930
end loading one data,time cost 1.654904
end loading one data,time cost 1.643668
end loading one data,time cost 1.663055
end loading one data,time cost 1.641192
end loading one data,time cost 1.652675
end loading one data,time cost 1.675678
end loading one data,time cost 1.642454
end loading one data,time cost 1.672420

In [ ]:
end loading one data,time cost 1.633667
end loading one data,time cost 1.608869
end loading one data,time cost 1.585573
end loading one data,time cost 1.603048
end loading one data,time cost 1.639884
end loading one data,time cost 1.633841
end loading one data,time cost 1.678963
end loading one data,time cost 1.641207
end loading one data,time cost 1.625344
end loading one data,time cost 1.623207
end loading one data,time cost 1.649238
end loading one data,time cost 1.635630
end loading one data,time cost 1.604664
end loading one data,time cost 1.610379
end loading one data,time cost 1.652588
end loading one data,time cost 1.597581
end loading one data,time cost 1.583224
end loading one data,time cost 1.629541
end loading one data,time cost 1.627871
end loading one data,time cost 1.657890
end loading one data,time cost 1.611370
end loading one data,time cost 1.586311
end loading one data,time cost 1.673285
end loading one data,time cost 1.651267
end loading one data,time cost 1.655988
end loading one data,time cost 1.631040
end loading one data,time cost 1.618624
end loading one data,time cost 1.654610
end loading one data,time cost 1.605169
end loading one data,time cost 1.590393
end loading one data,time cost 1.681857
end loading one data,time cost 1.626186
end loading one data,time cost 1.657457
end loading one data,time cost 1.642736
end loading one data,time cost 1.670448
end loading one data,time cost 1.646338
end loading one data,time cost 1.601079
end loading one data,time cost 1.588476
end loading one data,time cost 1.643233
end loading one data,time cost 1.688879
end loading one data,time cost 1.599864
end loading one data,time cost 1.585036
end loading one data,time cost 1.619789
end loading one data,time cost 1.589529
end loading one data,time cost 1.587214
end loading one data,time cost 1.607371